In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure,show,output_notebook
from bokeh.models import Legend
from bokeh.layouts import column
output_notebook()

Loading BokehJS ...

In [2]:
df=pd.read_csv("C:/Users/Administrator/Downloads/dataset.csv")
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [4]:
df['TimeStamp']=pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],dayfirst=True)
df['TimeStamp']

0       2016-10-04 07:59:00
1       2016-10-04 08:25:00
2       2016-10-04 08:59:00
3       2016-10-04 09:32:00
4       2016-10-04 09:59:00
                ...        
18363   2016-12-19 14:30:00
18364   2016-12-19 15:03:00
18365   2016-12-19 15:29:00
18366   2016-12-19 16:03:00
18367   2016-12-19 16:30:00
Name: TimeStamp, Length: 18368, dtype: datetime64[ns]

In [7]:
df['BasePrice']=10.0
df['Model_1_Price($)']=np.nan
df['Model_2_Price($)']=np.nan
df[['Model_1_Price($)','Model_2_Price($)']]

,Model_1_Price($),Model_2_Price($)
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
18363,NaN,NaN
18364,NaN,NaN
18365,NaN,NaN
18366,NaN,NaN


In [38]:
traffic_map = {'Low': 1, 'Medium': 2, 'High': 3}
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].map(traffic_map)

In [22]:
def baseline_linear_model(prev_price,occupancy,capacity,alpha=1.5):
    return prev_price + alpha * (occupancy / capacity)

In [39]:
def demand_base_model(base_price,occupancy,capacity,
                      queue,traffic,special_day,vehicle_weight,
                      alpha=0.6,beta=0.3,gamma=0.2,
                      delta=0.5,epsilon=0.7,lambd=0.5):
    
    demand=(
        alpha * (occupancy / capacity) +
        beta * queue -
        gamma * traffic + 
        delta * special_day +
        epsilon * vehicle_weight
    )
    
        
    normalized_dem=demand/10
    price=base_price * (1 + lambd * normalized_dem)
    price=np.clip(price, 0.5 * base_price, 2 * base_price)
    return price
    

In [40]:
vehicle_weight_map={'car':1.0, 'bike':0.5, 'truck':1.5}
df['VehicleWeight']=df['VehicleType'].map(vehicle_weight_map)
#df.drop(columns=['Price_Model1','Price_Model2','VehcileWeight'],inplace=True)
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,TimeStamp,BasePrice,Model_1_Price($),Model_2_Price($),VehicleWeight
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,NaN,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,10.0,NaN,NaN,1.0
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,NaN,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00,10.0,NaN,NaN,1.0
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,NaN,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00,10.0,NaN,NaN,1.0
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,NaN,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00,10.0,NaN,NaN,1.0
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,NaN,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00,10.0,NaN,NaN,0.5


In [41]:
df.isnull().sum()

ID                            0
SystemCodeNumber              0
Capacity                      0
Latitude                      0
Longitude                     0
Occupancy                     0
VehicleType                   0
TrafficConditionNearby    18368
QueueLength                   0
IsSpecialDay                  0
LastUpdatedDate               0
LastUpdatedTime               0
TimeStamp                     0
BasePrice                     0
Model_1_Price($)          18368
Model_2_Price($)          18368
VehicleWeight              1769
dtype: int64

In [42]:
df['VehicleWeight'].fillna(1.0,inplace=True)
df['VehicleWeight']

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3764\1659600792.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['VehicleWeight'].fillna(1.0,inplace=True)


0        1.0
1        1.0
2        1.0
3        1.0
4        0.5
        ... 
18363    1.5
18364    1.0
18365    1.0
18366    1.0
18367    1.0
Name: VehicleWeight, Length: 18368, dtype: float64

In [43]:
df.isnull().sum()

ID                            0
SystemCodeNumber              0
Capacity                      0
Latitude                      0
Longitude                     0
Occupancy                     0
VehicleType                   0
TrafficConditionNearby    18368
QueueLength                   0
IsSpecialDay                  0
LastUpdatedDate               0
LastUpdatedTime               0
TimeStamp                     0
BasePrice                     0
Model_1_Price($)          18368
Model_2_Price($)          18368
VehicleWeight                 0
dtype: int64

In [19]:
df.columns

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'TimeStamp',
       'BasePrice', 'Model_1_Price($)', 'Model_2_Price($)', 'VehicleWeight'],
      dtype='object')

In [44]:
Lot_IDs=df['ID'].unique()
price_tracking={}

for lot in Lot_IDs:
    lot_data=df[df['ID']==lot].copy().reset_index(drop=True)
    
    prev_price=10.0
    for i in range(len(lot_data)):
        occupancy=lot_data.loc[i,'Occupancy']
        capacity=lot_data.loc[i,'Capacity']
        queue=lot_data.loc[i,'QueueLength']
        traffic=lot_data.loc[i,'TrafficConditionNearby']
        special_day=lot_data.loc[i,'IsSpecialDay']
        vehicle_weight=lot_data.loc[i,'VehicleWeight']
        
        Model1Price=baseline_linear_model(prev_price,occupancy,capacity)
        lot_data.loc[i,'Model_1_Price($)']=Model1Price
        
        Model2Price=demand_base_model(10,occupancy,capacity,queue,traffic,special_day,vehicle_weight)
        lot_data.loc[i,'Model_2_Price($)']=Model2Price
        
        prev_price=Model1Price
        
    price_tracking[lot]=lot_data
    
print("All prices calculated successfully without errors.")

All prices calculated successfully without errors.


In [ ]:

plots=[]

for lot, lot_data in price_tracking.items():
    p=figure(title=f'Parking Lot {lot} - Price Over Time',
             x_axis_label='Time Step',
             y_axis_label='Price ($)',
             height=300,
             width=800
    )
    
    x=list(range(len(lot_data)))
    y_1=lot_data['Model_1_Price($)'].tolist()
    y_2=lot_data['Model_2_Price($)'].tolist()
    
    line_1=p.line(x, y_1, line_color='red',line_width=2,
                  legend_label='Model 1 : Baseline Linear Model')
    line_2=p.line(x, y_2, line_color='blue',line_width=2,
                  legend_label='Model 2 : Demand Based Model')
    
    p.legend.location='top_left'
    plots.append(p)
    
show(column(*plots))

In [ ]:
from geopy.distance import geodesic

def get_nearby_lots(curr_lat,curr_lon,all_lots,radius_km=1.0):
    nearby_lots=[]
    for lot in all_lots:
        lot_lan, lot_lon=lot['Latitude'], lot['Longitude']
        distance=geodesic((curr_lat, curr_lon), (lot_lan, lot_lon)).km
        
        if distance <= radius_km:
            nearby_lots.append(lot['ID'])
            
    return nearby_lots